In [ ]:
import sys
import os
import random
import datetime
import re
import math
import logging
from collections import OrderedDict
import multiprocessing
import numpy as np
import tensorflow as tf
import  keras
import keras.backend as K
import keras.layers as KL
import keras.engine as KE
import keras.models as KM

import index_tracker as track   
import csv
from keras.callbacks import LearningRateScheduler
import cv2
import scipy
import h5py
import skimage
import time

# set seed
from numpy.random import seed
seed(1234)
from tensorflow import set_random_seed
set_random_seed(1234)

from siammask import utils, rpn as rp, data_generator as dg, visualize, metrics
from siammask.config import Config


from siammask import siammask_model as modellib
import dicom_to_numpy as dtn
import matplotlib.pyplot as plt

from siammask import testing_code

**1. Set test hyperparameters**

In [ ]:
class TumorConfig(Config):    
    NAME = 'lung_tumor'    
    LOSS_WEIGHTS = {
        "rpn_class_loss": 1.,
        "rpn_bbox_loss": 1.,   
        "rpn_centerness_loss": 1.,        
        "rpn_mask_loss": 1.        
    }
    LEARNING_RATE = 0.001
    LEARNING_MOMENTUM = 0.9
    IMAGES_PER_GPU = 1
    TRAIN_BN = True
                 
    IMAGE_HEIGHT = 384
    IMAGE_WIDTH = 512
    IMAGE_SHAPE = [384, 512, 1]
    STEPS_PER_EPOCH = 1000
    VALIDATION_STEPS = 181
    
    BACKBONE_STRIDES = [8]
    RPN_ANCHOR_SCALES = [64]
    
    POST_NMS_ROIS_TRAINING = 1024
    POST_NMS_ROIS_INFERENCE = 512
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    RPN_NMS_THRESHOLD = 0.8
    BBOX_LOSS_LAMDA_D = 10
    BBOX_LOSS_LAMDA_C = 1
    
    TRAIN_MASKS_PER_IMAGE = 20
    
    USE_BN = True
    USE_DP = False
    
    ANCHORLESS = True 
    SIAMRPN = True 
    CROP = True
    
    MASK = False
    REFINE = False    
    STAGE4 = True
    SHARED = False
    BACKBONE = "resnet50"
    
    BBOX_RATIO = 0.7
    
    BBOX_SIGMA = 1.0
    
    KERNEL_IMAGE_SIZE = 128
    IMAGE_CHANNEL_COUNT = 3
    if CROP:
        FEATURES_SHAPE = [25, 25]
    else:
        FEATURES_SHAPE = [17, 17]   


# SET UP CONFIG
config = TumorConfig()

**2. Load test datasets**

In [ ]:
data_path = ''
patient_name = 'patient2'
model = 'siammask'
data_mode = 'decorrelated'
drr_data_path = data_path + patient_name + '/models/'+ model + '/' + data_mode +'/dataset/'
# Directory to save logs and trained model
LOG_DIR =  data_path + patient_name + '/models/' + model + '/' + data_mode +'/logs/'


gans=''
test_imgs = dtn.load_data(drr_data_path + 'test_imgs.hdf5', 'test_imgs')
test_labs = dtn.load_data(drr_data_path + 'test_labs.hdf5', 'test_labs')

**3. Preprocess loaded datasets**

In [ ]:
for i in range(test_labs.shape[0]):
    for j in range(test_labs.shape[1]):
        test_labs[i, j] = fit_ellipse(test_labs[i, j])
        
test_imgs = (test_imgs+1)/2.

**4. Load trained model**

In [ ]:
model = modellib.SiamMask(mode="inference", config=config,
                                      model_dir=LOG_DIR)


weights_path = model.find_last()
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True) 

**5. Test model on DRRs**

In [ ]:
# simulate test data
gt_bboxs, pred_bboxs = [], []      
gt_search_bboxs, pred_search_bboxs = [], []  
gt_masks, pred_masks = [], []   
 
for i in range(test_imgs.shape[0]):
    gt, pred = model.test(test_imgs[i], test_labs[i], verbose = 0)
    gt_bboxs.append(gt['gt_bboxs'])
    pred_bboxs.append(pred['pred_bboxs'][:, np.newaxis, :])
    gt_search_bboxs.append(gt['gt_search_bboxs'])
    pred_search_bboxs.append(pred['pred_search_bboxs'][:, np.newaxis, :])
    gt_masks.append(gt['gt_masks'])
    pred_masks.append(pred['pred_masks'])
gt_bboxs = np.stack(gt_bboxs, axis = 0)
pred_bboxs = np.stack(pred_bboxs, axis = 0)
gt_search_bboxs = np.stack(gt_search_bboxs, axis = 0)
pred_search_bboxs = np.stack(pred_search_bboxs, axis = 0)
gt_masks = np.stack(gt_masks, axis = 0)
pred_masks = np.stack(pred_masks, axis = 0)

gt_bboxs = np.reshape(gt_bboxs, (-1, 1, 4))
pred_bboxs = np.reshape(pred_bboxs, (-1, 1, 4)) 
gt_search_bboxs = np.reshape(gt_search_bboxs, (-1, 1, 4))
pred_search_bboxs = np.reshape(pred_search_bboxs, (-1, 1, 4))
gt_masks = np.reshape(gt_masks, (-1, 384, 512))
pred_masks = np.reshape(pred_masks, (-1, 384, 512))

**6. Compute metrics**

In [ ]:
miou = metrics.mIOU(gt_bboxs, pred_bboxs)
accuracy = metrics.accuracy(gt_bboxs, pred_bboxs, 
                            iou_threshold = 0.7)

cor_z, cor_x = metrics.concordance_correlation_coefficient(gt_search_bboxs, pred_search_bboxs)
z_mad, z_std, x_mad, x_std = metrics.mad(gt_bboxs, pred_bboxs)    
z_mad, z_std, x_mad, x_std = z_mad/3, z_std/3, x_mad/3, x_std/3 

**7. Load X-Rays**

In [ ]:
data_path = ''
patient_name = 'patient2'
xray_data_path = data_path + patient_name + '/X_rays/'

xrays = load_data(xray_data_path + 'patient2_x_rays_1.h5', 'img')
angles = load_data(xray_data_path + 'patient2_x_rays_1.h5', 'rot')
angles = angles-270

**8. Preprocess X-Rays**

In [ ]:
xrays = utils.log_transform(xrays)

shape = np.array(xrays.shape[1:])
new_shape = shape//2
xrays = xrays[:, shape[0]//2-new_shape[0]//2:shape[0]//2+new_shape[0]//2, 
                shape[1]//2-new_shape[1]//2:shape[1]//2+new_shape[1]//2] 

xrays = (xrays-np.min(xrays))/(np.max(xrays)-np.min(xrays))
xrays = utils.gamma_transform(xrays, 5)

**9. Test model on X-Rays**

In [ ]:
outputs_xrays = model.test(xrays[0:], labs=None, verbose = 0)

pred_masks = np.round(outputs_xrays[1]['pred_masks']).astype('uint8')
pred_bboxs = outputs_xrays[1]['pred_bboxs'][np.newaxis, :, np.newaxis, :]
pred_scores = outputs_xrays[1]['pred_scores']

pred_centers =  np.array([pred_bboxs[..., 0] + (pred_bboxs[..., 2] -  pred_bboxs[..., 0])//2,  pred_bboxs[..., 1] + (pred_bboxs[..., 3] -  pred_bboxs[..., 1])//2])    
pred_centers = np.transpose(pred_centers[:, 0, :, 0], (1, 0)) 